In [ ]:
#default_exp test.find_orders

# Find Orders

> Convenience endpoint to find order examples in Redis database

In [ ]:
#export
from typing import Dict, List
from datetime import datetime, timezone, timedelta
import os
import json
import pickle
import aioredis
from dataclasses import dataclass
from hopeit.dataobjects import dataobject
from hopeit.server.serialization import serialize, Serialization, deserialize
from hopeit.server.compression import Compression
from hopeit.app.context import EventContext, PostprocessHook
from hopeit.app.api import event_api
from hopeit.app.logger import app_logger

from fraud_poc.live.predict import OrderInfo

In [ ]:
#export
__steps__ = ['scan_db']

__api__ = event_api(
    summary="Test: Find Orders",
    query_args=[
        ("prefix", str, "Prefix for customer_id or email with * as a wildcard"),
        ("num_items", int, "Number of items to retrieve")
    ],
    responses={
        200: (List[OrderInfo], "list of orders")
    }
)

logger = app_logger()

db = None
cur = b'0'


In [ ]:
#export
async def scan_db(payload: None, context: EventContext, prefix: str, num_items: int):
    address = context.env['db']['url']
    logger.info(context, f"Connecting to database {address}...")
    results = []
    redis = await aioredis.create_redis(address)
    cur = b'0'  # set initial cursor to 0
    for i in range(int(num_items)):
        cur, keys = await redis.scan(cur, match=f'{prefix}*')
        if cur is None: break
    print(keys)
    for k in keys:
        try:
            item = await redis.get(k)            
            item = deserialize(item, Serialization.PICKLE4, Compression.LZ4, dict)
            results.append(OrderInfo.from_dict({'location_lat': 0.0, 'location_long': 0.0, **item}))
        except Exception as e:
            pass # Ignoring items that are not orders, this is just a test method
    return results

### Test from notebook

In [ ]:
from hopeit.testing.apps import config, execute_event
from fraud_poc.live.predict import OrderInfo
import uuid
 
def new_key(): return str(uuid.uuid4())

app_config = config('config/fraud-service.json')
result = await execute_event(app_config, 'test.find_orders', None, prefix='*', num_items=10)
result

2020-07-08 14:47:56,971 | INFO | fraud-poc 0.0.1-service test.find_orders leos13 27404 | Connecting to database redis://localhost:6379... | track.operation_id=test_operation_id | track.request_id=test_request_id | track.request_ts=2020-07-08T14:47:56.971741+00:00


[b'7545576ffe1b7c1d9d8d2e82d0191fa057df695f', b'71264e73-3061-4990-8297-6c09314b1a9d', b'b1f59a4bd99f595478e4259a6e3576d1081cbee8', b'fraud_poc.0x0x1_training.data.feature_calc', b'd16ed4b6-6e6f-4788-b45b-acd6609d7649', b'fraud_poc.0x0x1_training.data.make_sample_data.submit_job', b'fc55c49c955c439eff004e08ddc5c35e59023a6a', b'a7163f535fb73b3f8de90ca74387b22dc68276c3', b'63f6259e971adaf70ad742be1a1707b48712c02b', b'0ccf564669e982312eec408bb103ae929a01c7e6']

{'order_id': 'ce4798f5-6127-4d6e-bf1d-dda810eab26b', 'order_date': '2020-07-07T06:33:18+00:00', 'customer_id': '271d8c5e-e4e3-4377-a3e3-673ccf153664', 'ip_addr': 'f95e9c978b7f88dde5b9eb39417070251603db2d', 'order_amount': 916.7097195892065, 'email': '7545576ffe1b7c1d9d8d2e82d0191fa057df695f', 'customer_id_by_email': '["271d8c5e-e4e3-4377-a3e3-673ccf153664", "271d8c5e-e4e3-4377-a3e3-673ccf153664", "271d8c5e-e4e3-4377-a3e3-673ccf153664", "271d8c5e-e4e3-4377-a3e3-673ccf153664", "271d8c5e-e4e3-4377-a3e3-673ccf153664", "271d8c5e-e4e3-43

[OrderInfo(order_id='ce4798f5-6127-4d6e-bf1d-dda810eab26b', customer_id='271d8c5e-e4e3-4377-a3e3-673ccf153664', order_date=datetime.datetime(2020, 7, 7, 6, 33, 18, tzinfo=tzutc()), email='7545576ffe1b7c1d9d8d2e82d0191fa057df695f', ip_addr='f95e9c978b7f88dde5b9eb39417070251603db2d', order_amount=916.7097195892065, location_lat=0.0, location_long=0.0),
 OrderInfo(order_id='eca4330a-9f6e-44dd-aea8-f8d7c54c8950', customer_id='71264e73-3061-4990-8297-6c09314b1a9d', order_date=datetime.datetime(2020, 7, 4, 23, 27, 45, tzinfo=tzutc()), email='9fc4829456e3666af2644e22dda27036f1bd95c6', ip_addr='68bf5c2a7d0cc0107eebaaf105ecef8faf51b812', order_amount=824.7756099864736, location_lat=0.0, location_long=0.0),
 OrderInfo(order_id='0f4de1e7-ae83-47fb-a45c-98da4e399b4b', customer_id='1faf28b9-0dff-47f1-bcd8-689053805b16', order_date=datetime.datetime(2020, 7, 6, 10, 59, 28, tzinfo=tzutc()), email='b1f59a4bd99f595478e4259a6e3576d1081cbee8', ip_addr='95c99f65815f00f09a3fb62f129f4a80adce6301', order_amo